# Uso de MyFitnessPal para conseguir la información nutricional de mi comida

Vamos a utilizar la librería de Python que tiene OpenAI que nos facilita la vida para utilizar uno de los modelos de ChatGPT.

In [28]:
import requests
import os
from openai import OpenAI

Para esta demo, necesito dos api keys, una es la de Open AI, que me permite hacer peticiones a la API.
La otra es la de Rapid API que viene muy bien para hacer demos o pruebas, tenéis a disposición múltiples apis y muchas son gratuitas o tienen un tier gratuito. Esto nos permite probar cosas sin tener que pagar o registrarnos en todas.
El siguiente código carga mi fichero con las variables de entorno `.env` donde tengo metidas las dos api keys que necesito.

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

Cargamos las variables de entorno para poder utilizarlas más cómodamente. Creamos el `client` de la librería de OpenAI. Por defecto, utilizará OPENAI_API_KEY cuando necesito la api key para conectarse. Si lo tenéis metido en otra variable, habrá que pasárselo como parámetro.

In [30]:
# Set your OpenAI and MyFitnessPal API keys
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
RAPIDAPI_KEY = os.environ.get('RAPIDAPI_KEY')

client = OpenAI()

Vamos a escribir primero una función para contactar con la api de MyFitnessPal y recuperar la información nutricional de un alimento.
Esta función es muy sencilla, hace una petición a la api de MyFitnessPal para buscar por palabra clave. Devuelve un array de jsons con los resultados que ha encontrado. Aquí vamos a utilizar por defecto el primer resultado.

In [31]:
def fetch_nutrition_data(keyword):
    url = "https://myfitnesspal2.p.rapidapi.com/searchByKeyword"
    headers = {
        "x-rapidapi-host": "myfitnesspal2.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY,        
    }
    params = {
        "keyword": keyword,
        "page": 1
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API Error: {response.status_code} - {response.text}"}

Por último, necesitamos una función para contactar con chat gpt. El primer prompt que le vamos a pasar a chat gpt es con el `role: system`. Este le dará a chat gpt unas instrucciones de lo que esperamos del modelo. Qué queremos que haga? Qué debe hacer con lo que le diga el usuario? En este caso utilizamos un texto muy corto. Veréis que no hace falta escribir textos muy largos para conseguir buenos resultados rápidamente pero si queréis hacer cosas más avanzadas, quizás necesitéis darle alguna vuelta.

El otro prompt explica a 

In [32]:
def parse_food_items(user_input):
    gpt_prompt = ("""
        You are a food item extractor. Break down the user's input into a list of food items 
        with quantities and descriptions. For example:\n\n
        Input: 'I ate 2 oreos and a glass of milk'
        Output: [{'quantity': '2, 'item': 'oreos'}, {'quantity': 1, 'item': 'glass of milk'}]\n\n
        """
    )

    gpt_prompt += f"Input: '{user_input}'. \nOutput:"
        
    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that helps summarize nutritional information."
            },
            {
                "role": "user",
                "content": user_input
            }
        ]
    )
    
    message = chat_completion.choices[0].message.content
    
    return message

Ahora viene lo que es la interacción con el modelo. Primero pediremos al usuario que nos 

In [35]:
user_input = input("What are you eating?").strip()

What are you eating? I am eating a sandwich of ham and cheese and three snack bars of chocolate


In [36]:
food_items = parse_food_items(user_input)
# food_items = [{"quantity": 2, "item": "oreos"}, {"quantity": 1, "item": "cup of apple juice"}]

print(food_items)

To summarize the nutritional information for your meal, we can break it down into two components: the ham and cheese sandwich and the chocolate snack bars.

1. **Ham and Cheese Sandwich**: 
   - **Ham**: Typically high in protein but can also contain sodium and fat. The nutritional values can vary based on the type of ham (e.g., turkey, honey-glazed, or regular).
   - **Cheese**: Generally provides protein and calcium, but can also be high in saturated fats and sodium. Common types like cheddar or Swiss have different calorie counts.
   - **Bread**: Provides carbohydrates and fiber, though the exact amounts depend on the type of bread used (white, whole wheat, etc.).

2. **Chocolate Snack Bars**: 
   - These often contain sugars, fats, and carbohydrates. The specific nutritional values depend on the brand and ingredients used. Typically, they can be calorie-dense, with added sugars and some protein if they contain nuts or protein sources.

### Estimated Nutritional Breakdown (General R